# 基于SQuAD的数据集微调 Bert模型

In [12]:
import os
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

from datasets import load_dataset

# 使用SQuAD数据集来微调
datasets = load_dataset("squad_v2")

In [13]:
from transformers import AutoTokenizer,BertForQuestionAnswering

# 加载分词器和模型
tokenizer =  AutoTokenizer.from_pretrained("bert-base-uncased",use_fast=True)
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


- 下面开始对数据开始预处理

In [14]:
def preprocess_data(examples):
# 1. 正常分词，使用滑动窗口
    inputs = tokenizer(
        examples['question'], 
        examples['context'], 
        truncation="only_second", 
        padding='max_length', 
        max_length=384,
        stride=128,
        return_offsets_mapping=True,
        return_overflowing_tokens=True,
    )

    sample_mapping = inputs.pop("overflow_to_sample_mapping")
    offset_mapping = inputs.pop("offset_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        sample_index = sample_mapping[i]
        input_ids = inputs['input_ids'][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        
        # 找到序列中 Context 的区间（区分 Question 和 Context）
        sequence_ids = inputs.sequence_ids(i)
        
        # 答案信息
        answer = examples['answers'][sample_index]
        
        # 如果没有答案，直接指向 [CLS]
        if len(answer['answer_start']) == 0:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            start_char = answer['answer_start'][0]
            end_char = start_char + len(answer['text'][0])
            
            # 找到当前分片在原始 Context 中的起始和结束 Token 索引
            token_start_index = 0
            token_end_index = 0
            
            # 找到 Context 在 input_ids 中的起始和结束位置
            token_search_start = 0
            while sequence_ids[token_search_start] != 1:
                token_search_start += 1
                
            token_search_end = len(input_ids) - 1
            while sequence_ids[token_search_end] != 1:
                token_search_end -= 1
            
            # 判断答案是否完全在当前分片的 Context 范围内
            if not (offsets[token_search_start][0] <= start_char and offsets[token_search_end][1] >= end_char):
                # 答案不在这个分片里，设为 [CLS]
                start_positions.append(cls_index)
                end_positions.append(cls_index)
            else:
                # 答案在分片里，开始精确定位
                curr = token_search_start
                while curr <= token_search_end and offsets[curr][0] <= start_char:
                    curr += 1
                start_positions.append(curr - 1)
                
                curr = token_search_end
                while curr >= token_search_start and offsets[curr][1] >= end_char:
                    curr -= 1
                end_positions.append(curr + 1)

    inputs['start_positions'] = start_positions
    inputs['end_positions'] = end_positions 
    return inputs

# 应用预处理函数
tokenized_dataset = datasets.map(
    preprocess_data, 
    batched=True, 
    remove_columns=datasets["train"].column_names
)

In [15]:
# 对数据集进行划分
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["validation"]

In [16]:
import torch
import sys

print(f"Python 版本: {sys.version}")
print(f"PyTorch 版本: {torch.__version__}")
print(f"CUDA 是否可用: {torch.cuda.is_available()}")
if not torch.cuda.is_available():
    import ctypes
    # 尝试加载 nvcuda.dll (Windows) 看看驱动库是否存在
    try:
        ctypes.WinDLL('nvcuda.dll')
        print("NVIDIA 驱动库 (nvcuda.dll) 检测正常")
    except Exception as e:
        print(f"无法加载 NVIDIA 驱动库: {e}")

Python 版本: 3.12.12 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 20:05:38) [MSC v.1929 64 bit (AMD64)]
PyTorch 版本: 2.6.0+cu124
CUDA 是否可用: True


In [17]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./bert-qa",
    eval_strategy="epoch",
    learning_rate=3e-5,         # 学习率
    per_device_train_batch_size=8, # 训练批次大小
    per_device_eval_batch_size=8,  # 评估批次大小
    num_train_epochs=2,
    weight_decay=0.01,  
    )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)
# 开始训练
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

In [ ]:
question ="What is the name of the repository for pre-trained transformers? "
context = "Transformers provides thousands of pretrained models to perform tasks on different modalities such as text, vision, and audio. The library is maintained by the Hugging Face team and the open-source community. The models can be easily integrated into applications using the Transformers API and are available through the Hugging Face Model Hub."

inputs = tokenizer(question, context, return_tensors="pt")
outputs = model(**inputs)

start_index=torch.argmax(outputs.start_logits)
end_index=torch.argmax(outputs.end_logits)+1
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_index:end_index]))
print(f"Answer: {answer}")

由于huggingface并不会添加vocab_file在最后的权重文件当中，因此需要重新添加

In [ ]:
import os
from transformers import AutoTokenizer

# 你的模型保存根目录
model_path = "../bert-qa/checkpoint-32940"   # 确认是这个路径

# 第一步：查看当前目录有哪些文件
print("修复前目录内容：")
print(os.listdir(model_path))

# 第二步：从 bert-base-uncased 下载并复制 tokenizer 文件到你的目录
print("\n正在从 bert-base-uncased 复制 tokenizer 文件（vocab.txt 等）...")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizer.save_pretrained(model_path)

# 第三步：查看修复后文件
print("\n修复完成！现在目录内容：")
print(os.listdir(model_path))

# 你应该会看到多了这些文件：
# vocab.txt
# tokenizer_config.json
# special_tokens_map.json
# tokenizer.json（可选）

修复前目录内容：
['config.json', 'model.safetensors', 'optimizer.pt', 'rng_state.pth', 'scheduler.pt', 'trainer_state.json', 'training_args.bin']

正在从 bert-base-uncased 复制 tokenizer 文件（vocab.txt 等）...

修复完成！现在目录内容：
['config.json', 'model.safetensors', 'optimizer.pt', 'rng_state.pth', 'scheduler.pt', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'trainer_state.json', 'training_args.bin', 'vocab.txt']


In [19]:
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
import torch
import os

# 直接使用根目录！（最保险的方式）
model_path = "../bert-qa/checkpoint-32940"   

# 加载 tokenizer 和 model
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForQuestionAnswering.from_pretrained(model_path, local_files_only=True)

# 创建问答 pipeline
qa_pipeline = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1  # 有GPU用GPU，没有用CPU
)

# 测试你的 fine-tune 模型
question = "What is the capital of France?"
context = "France is a country in Europe. The capital of France is Paris, which is known for its art, fashion, and culture."
result = qa_pipeline(question=question, context=context)

print("\n=== 测试结果 ===")
print("问题：", question)
print("答案：", result['answer'])
print("置信度：", f"{result['score']:.4f}")

Device set to use cuda:0



=== 测试结果 ===
问题： What is the capital of France?
答案： Paris
置信度： 0.9952


## 总结
上面是使用数据集对bert进行了微调，其中涉及到了NLP中的分词、模型加载、tokenizer的创建、测试、评估的过程，是一个最小的微调示例，后续会在其中加入其他优化过程。
其中，bert模型是一个英文语言训练模型，对英文问题的理解能力更好，中文则不行，可以尝试去改q&c，就会发现置信度的变化。

[详细可以看bert_url](https://huggingface.co/google-bert/bert-base-uncased)